<a href="https://colab.research.google.com/github/dkovah/tcbscans2pdf/blob/main/TCB_Offline_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1- Import stuff

In [1]:
from bs4 import BeautifulSoup
from PIL import Image
import requests
from io import BytesIO
import re
from zipfile import ZipFile

2- Edit download parameters

In [2]:
# Manga URL on TCB website
base_url = 'https://tcbscans.com/mangas/5/one-piece'

# First chapter to download
first = 1093

# Last chapter to download
last = 1095

# If you are running the notebook locally, set the path to save the pdf files.
# Leave it blank if you are running this on Colab
save_path = ""

3- Fetch chapter links

In [3]:
response = requests.get('https://tcbscans.com/mangas/5/one-piece')

4- Parse chapter links

In [4]:
manga_name = base_url.split("/")[-1]
soup = BeautifulSoup(response.content, 'html.parser')
anchors = soup.findAll('a', { 'class': 'block border border-border bg-card mb-3 p-3 rounded'})
numbers = [i for i in range(first, last + 1)]

def select_anchors(anchor):
  parts = anchor.attrs["href"].split("/")[-1].split('-')
  chapter_number = parts[-3] if parts[-2] == "review" else parts[-1]
  if int(chapter_number) in numbers:
    return anchor.attrs["href"]

def get_link(anchor):
  return "https://tcbscans.com" + anchor.attrs["href"]

selected_anchors = list(filter(select_anchors, anchors))
selected_links = list(map(get_link, selected_anchors))

5- Fetch chapter data

In [5]:
chapter_list = []
for link in selected_links:
  response = requests.get(link)
  soup = BeautifulSoup(response.content, 'html.parser')
  pictures = soup.findAll('img', alt=re.compile(r'Page'))
  pictures = list(map(lambda p : p.attrs["src"], pictures))
  chapter_list.append(pictures)
chapter_list.reverse()

6- Fetch images and generate PDF (if it fails, try again)

In [7]:
for index, chapter in enumerate(chapter_list):
  image_list = []
  for image_url in chapter:
    response = requests.get(image_url)
    png = Image.open(BytesIO(response.content))
    img = Image.new("RGB", png.size, (255, 255, 255))
    img.paste(png)
    image_list.append(img)
  filename = save_path + str(numbers[index]) + ".pdf"
  image_list[0].save(filename, "PDF" ,resolution=100.0, save_all=True, append_images=image_list[1:])

7- If you are running this locally, PDF files should be on the specified path, else they should available to download on the "files" tab at the left

8- (Optional) If you are using Colab, zip all chapters for an easier download

In [8]:
with ZipFile('chapters.zip', 'w') as myzip:
    for n in numbers:
      myzip.write(str(n)+'.pdf')